<a href="https://colab.research.google.com/github/kconstable/market_predictions/blob/main/market_data_av.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
# import sys
# !{sys.executable} -m pip install plotly

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
import pickle
import re
from datetime import date, timedelta,datetime
from tabulate import tabulate
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from google.colab import files
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from google.colab import drive
drive.mount('/content/drive')


# alphavalue key
with open('/content/drive/MyDrive/Colab Notebooks/data/av_key.txt') as f:
    key = f.read().strip()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Economic and Technical Indicators

## Economic Indicators
Pull economic indicator data from Alpha Advantage

In [ ]:
def get_economic_indicators(funct,key,interval=None,maturity=None,throttle=0):
  """
  Returns Economic Indicator Data with missing values interpolated between dates
  Monthly Data:
    NONFARM_PAYROLL, INFLATION_EXPECTATION,CONSUMER_SENTIMENT,UNEMPLOYMENT
  Daily, Weekly, Monthly Data:  
    FEDERAL_FUNDS_RATE = interval (daily,weekly,monthly)
    TREASURY_YIELD = interval (daily, weekly, monthly), 
                     maturity (3month, 5year, 10year, and 30year)
  """
  
  # query strings
  # Monthly Data:
  if funct in ['NONFARM_PAYROLL','INFLATION_EXPECTATION','CONSUMER_SENTIMENT','UNEMPLOYMENT']:
    url = f'https://www.alphavantage.co/query?function={funct}&apikey={key}'

  # Daily, Weekly or Monthly Data:
  # Interest Rates
  if funct == 'FEDERAL_FUNDS_RATE':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&apikey={key}'

  # Treasury Yield  
  if funct == 'TREASURY_YIELD':
    url = f'https://www.alphavantage.co/query?function={funct}&interval={interval}&maturity={maturity}&apikey={key}'

  # pull data
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # convert to df
  df = pd.DataFrame(d['data'])

  # move date to a datetime index
  df.date = pd.to_datetime(df.date)
  df.set_index('date',inplace=True)

  # add the ticker name and frequency
  df['name'] = d['name']
  df['interval']=d['interval'] 

  # clean data & interpolate missing values
  # missing data encoded with '.'
  # change datatype to float
  df.replace('.',np.nan,inplace=True)
  df.value = df.value.astype('float')

  # missing data stats
  missing =sum(df.value.isna())
  total =df.shape[0]
  missing_pct = round(missing/total*100,2)

  # interpolate using the time index
  if missing >0:
    df.value.interpolate(method='time',inplace=True)
    action = 'interpolate'
  else:
    action = 'none'

  # Print the results
  if maturity is not None:
    summary = ['Economic Indicator',funct+':'+maturity,str(total),str(missing),str(missing_pct)+'%',action]
  else:
    summary = ['Economic Indicator',funct,str(total),str(missing),str(missing_pct)+'%',action]


  return {'summary':summary,'data':df}


## Technical Indicators
Pull technical indicator data from Alpha Advantage. Only works for stocks, not crypto.

In [ ]:
def get_technical_indicators(symbol,funct,key,interval,time_period=None,throttle=0):
  """
  Returns Technical Indicators (only works for stocks, not cyrpto)
  MACD:   symbol,interval
  RSI:    symbol,interval,time_period
  BBANDS: symbol,interval,time_period

  Parameters:
          interval: (1min, 5min, 15min, 30min, 60min, daily, weekly, monthly)
          series_type: (open, close,high,low)-default to close
          timer_periods: Integer
  """
  # build the query string
  if funct =='MACD':
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&apikey={key}'
  if funct in ['RSI','BBANDS']:
    url = f'https://www.alphavantage.co/query?function={funct}&symbol={symbol}&interval={interval}&series_type=close&time_period={time_period}&apikey={key}'

  # request data as json, convert to dict, pause request to avoid the data throttle
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract to a df, add the indicator name, convert the index to datetime
  df = pd.DataFrame(d[f'Technical Analysis: {funct}']).T
  df.index = pd.to_datetime(df.index)

  # convert the data to float
  for col in df.columns:
    df[col] = df[col].astype('float')

  # check for missing data
  missing = df.isnull().any().sum()
  total = len(df)
  missing_pct = round(missing/total*100,2)


  # Print the results
  summary=['Technical Indicator',funct,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}


# Visualizations
Functions to plot prices, economic and technical indicators for stocks and crypto.
+ Candlestick Plots + Economic Indicators
+ Technical Indicators + Interest Rates
+ Correlations between Features
+ Distribution of prices

### Candlestick/Economic Indicators

In [ ]:
def plot_candlestick(df,yearfrom=None):
    """
    Plots candleskick plots: open/close/high/low prices + volume
    Input:
      df      : A dataframe with the price data
      yearfrom: filters the df by year
    """

    # get ticker name
    name = df.symbol.unique()[0]

    # filter by year if provided
    if yearfrom is not None:
      df = df[df.index.year >= yearfrom]
    
    # Create subplots
    fig = make_subplots(rows=6, cols=1, 
                        shared_xaxes=True, 
                        subplot_titles=('Prices: Open/High/Low/Close','Volume','Non-Farm Payroll','Consumer Sentiment','Expected Inflation','Unemployment'),
                        vertical_spacing=0.07)
                   
    # OHLC Plot
    fig.add_trace(go.Candlestick(x=df.index, open=df.open, high=df.high,low=df.low, close=df.close, 
                name="Price",
                increasing_line_color= '#42BDA0', 
                decreasing_line_color= '#BD425F'), 
                row=1, col=1)
    
    
    # area chart for volume
    fig.add_trace(go.Scatter(x=df.index, y=df.volume, 
                fill='tozeroy',
                name='Volume',
                mode='none',
                fillcolor='rgba(156,189,66,.5)'),
                row=2,col=1)
    
    # Non-Farm Payrolls
    fig.add_trace(go.Scatter(x=df.index, y=df.nfp,name='Non-Farm Payroll',line_color='#BD425F'),
                  row=3,col=1)
    
    # Consumer Sentiment
    fig.add_trace(go.Scatter(x=df.index, y=df.cs,name='Consumer Sentiment',line_color='#42BDA0'),
                  row=4,col=1)

    # Expected Inflation    
    fig.add_trace(go.Scatter(x=df.index, y=df.infl,name='Expected Inflation',line_color='#6342BD'),
                  row=5,col=1)
    
    # Unemployment    
    fig.add_trace(go.Scatter(x=df.index, y=df.unemployment,name='Unemployment'),
                  row=6,col=1)

    # Do not show OHLC's rangeslider plot 
    fig.update(layout_xaxis_rangeslider_visible=False)
    fig.update_layout(template='plotly_white',
                      title = f"Price, Volume and Economic Indicators: {name}",
                      width = 1000,
                      height = 800)
          
    fig.show()

### Technical Indicators

In [ ]:
def plot_technicals(df,yearfrom=None):
  """
  Plots technical indicators
  Input:
    df: a dataframe with techical indicators
    yearfrom: filters the df by year
  """

  # get the symbol name
  name = df.symbol.unique()[0]

  # filter by year if provided
  if yearfrom is not None:
    df = df[df.index.year >= yearfrom]


  # make subplots
  fig = make_subplots(rows=2, cols=2,
                      shared_xaxes=True,
                      subplot_titles = ('Bollinger Bands','MACD','Interest Rates','RSI'),
                      vertical_spacing = 0.07)

  # boolinger bands
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['b-upper'],name='Bollinger-Upper',line_color="#DFEBF9"),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df['b-lower'],name='Bollinger-Lower',line_color='#DFEBF9',fill='tonexty'),row=1,col=1)
  fig.add_trace(go.Scatter(x=df.index,y=df['b-middle'],name='Closing Price',line_color='#297AD6'),row=1,col=1)


  # MACD
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['macd'],name='MACD',line_color='#F7DAC6'),row=1,col=2)
  fig.add_trace(go.Scatter(x=df.index,y=df['macd_signal'],name='MACD Signal',line_color='#E68A4C'),row=1,col=2)
  try:
    fig.add_trace(go.Bar(x=dff.index,y=df['macd_hist'],name='MACD Hist',marker_color='#E06D1F', marker_line_color='#E06D1F'),row=1,col=2)
  except:
    # skip mcad histogram for crypto
    print('')


  # Interest Rates
  # add Fed Rate, then loop through the df columns to add any feature that is a yield
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index, y=df.ir,name='Fed Funds Rate',line_color='#4B9D0C'),row=2,col=1)
  colors = ['#0E340F','#29541F','#5E9C53','#A3E090','#537455','#3B463E']
  rate_cols = [c for c in df.columns if 'yield' in c]
  for i,rate in enumerate(rate_cols):
    # Extract the name of the yield
    name = 'Yield:'+re.findall(r'\d+', rate)[0]+' Year'
    fig.add_trace(go.Scatter(x=df.index, y=df[rate],name=name,line_color = colors[i]),row=2,col=1)



  # # RSI
  # ********************************************************************************************
  fig.add_trace(go.Scatter(x=df.index,y=df['rsi'],name='RSI',line_color='#F00F3C'),row=2,col=2)
  fig.add_shape(type ='rect',
                x0=min(df.index),
                x1=max(df.index),
                y0=30.0,
                y1=70.0,
                line=dict(color='#F00F3C'),
                fillcolor='#F00F3C',
                opacity=0.1,
                row=2,col=2)
  fig.update_shapes(dict(xref='x',yref='y'),row=2,col=1)


  # Set template
  fig.update_layout(template = 'plotly_white',width= 1000,height=800,title ='Technical Indicators & Yields')
  fig.show()

### Distributions

In [ ]:
def plot_distribution(df,feature):
  """
  Plot a historgram and bloxplot the input feature
  Input: 
    df:      a dataframe of features
    feature: a feature name
  Output:
    Plots the histogram and boxplot for the feature
  """

  # create subplots
  fig = make_subplots(rows=1, cols=2)

  # histogram
  fig.add_trace(
      go.Histogram(x=df[feature],name='histogram'),
      row=1, col=1
  )

  #box plot
  fig.add_trace(
      go.Box(y=df[feature],name='boxplot'),
      row=1, col=2
  )

  #
  fig.update_layout(height=500, 
                    width=500, 
                    title_text=f"Distribution:{feature}",
                    template='plotly_white')
  fig.show()




### Correlations

In [ ]:
def plot_corr(df):
  """
  Plots correlations in a heatmap
  Input: 
    df: a dataframe of features to plot
  """
  # subset columns
  # remove redundent columns (only keep one price, and the signal for technical indicators)
  cols = [c for c in df.columns if c not in ['open','high','low','b-upper','b-lower','macd_hist','macd','symbol']]
          
  # calculate correlations
  cm =df[cols].corr()

  # plot a heatmap of correlations
  fig = go.Figure()
  fig.add_trace(go.Heatmap(
        z = cm,
        x = cm.columns.values,
        y = cm.columns.values,
        colorscale = 'Bluyl'
      )
  )
  fig.update_layout(
      title_text="Feature Correlations", 
      title_x=0.5, 
      width=800, 
      height=800,
      yaxis_autorange='reversed',
      template = 'plotly_white'
  )

  fig.show()


# Stocks

### Get Stock/Commodity Data
Pull stock/commodity data from Alpha Advantage

In [ ]:
def get_ticker_data(symbol,key,outputsize='compact',throttle=0):
  """
  Returns daily data for a stock (symbol)
    outputsize: compact(last 100) or full (20 years)
    key: apikey
    symbols: OILK (oil ETF),BAR(gold ETF),VXZ (volatility ETF)
  """
  url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={outputsize}&apikey={key}'
  r = requests.get(url)
  time.sleep(throttle)
  d = r.json()

  # extract data to a df
  df = pd.DataFrame(d['Time Series (Daily)']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Symbol']

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # Calculate missing data
  missing = sum(df.close.isna())
  total = df.shape[0]
  missing_pct = round(missing/total*100,2)

  # Print the results
  summary = ['Ticker',symbol,str(total),str(missing),str(missing_pct)+'%','none']

  return {'summary':summary,'data':df}

## Combine Stock Data
+ Daily Price/commodity data, monthly economic indicators, and daily technical indicators from Alhpa Advantaga are combined into a consolidated dataset
+ Some economic indicators were missing data.  Given that the data is time-series based, missing data points were linearly interpolated
+ The get_consolidated_stock_data function performs these functions and returns a consolidated data frame free of missing data

In [ ]:
def get_consolidated_stock_data(symbol,key,config,outputsize='compact',throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: stock ticker
    key   : api key
    config: dictionary which lists the economic, technical and commodities to pull
    outputsize: compact(latest 100) or full (up to 20 years of daily data)
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get stock prices
  try:
    results  = get_ticker_data(symbol,key,outputsize,0)
    dff = results['data']
    summary.append(results['summary'])
    print(f'Complete:===>Ticker:{symbol}')
  except:
    print(f'Error:===>Ticker:{symbol}')



  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,outputsize,throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  for indicator,values in config['Technical'].items():
    try:
      results = get_technical_indicators(symbol,indicator,key,values['interval'],values['time_period'],throttle)
      df = results['data']
      summary.append(results['summary'])

      dff = dff.join(df,how='left')
      print(f"Complete:===>{indicator}")
    except:
      print(f"Error===>{indicator}")

  
  # clean column names
  dff.rename(columns={"Real Upper Band":'b-upper',
                      "Real Lower Band":'b-lower',
                      "Real Middle Band":"b-middle",
                      "RSI":"rsi",
                      "MACD_Hist":"macd_hist",
                      "MACD_Signal":"macd_signal",
                      "MACD":"macd"
                      },inplace=True)
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

## Pull Stock Data


A selection of economic and technical indicators were selected as possible inputs to the LSTM model. Data was pulled for BLX.TO (Boralex) which is a Canadian clean energy company, and VM Ware, which is a US based cloud computing company.  The economic & technical indicators for both stocks were the same, but different commodities and market index featues were used in each to reflect the different industries in which the companies operate. 
+ **Economic Indicators**
  + Interest Rates: The Fed interest rate, short, middle and long term bond yields
  + [Non-farm payrolls](https://www.investopedia.com/terms/n/nonfarmpayroll.asp) - the number of people currently employed in the US (monthly indicator)
  + [Unemployment rate](https://www.investopedia.com/terms/u/unemploymentrate.asp) - the percentage of people unemployed in the US
  + [Consumer sentiment](https://www.investopedia.com/terms/c/cci.asp) -measure of optimistic/pessimistic consumers based on survey data
  + [Inflation expection](https://fred.stlouisfed.org/series/MICH) -Median expected price change next 12 months, based on consumer surveys
+ **Commodities and Market Indexes**
  + **VM Ware & Boralex**
    + S&P500 Index (The SPY ETF is used as a proxy)
    + Gold Prices (The GLD ETF is used as a proxy)
    + Oil Prices (The OIL ETF is used as a proxy)
    + Market Volatility Index (The VXX ETF is used as a proxy)
  + **Boralex**
    + Energy Index (oil + gas) (The XLE ETF is used as a proxy)
    + Clean Energy Index (The PBD ETF is used as a proxy)
    + The Canadian Stock market index (The ECW ETF is used as a proxy)
  + **VM Ware**
    + The Nasdaq Index (heavy tech companies) (The QQQ ETF is used as a proxy)
    + The tech sector in the US (The VGT ETF is used as a proxy)
    + The cloud computing sector (The SKYY ETF is used as a proxy)
+ **Technical Indicators**
  + Bollinger Bands
  + RSI (relative strenth indicator)
  + MACD (moving average convergence/divergence)
+ **Seasonal**
  + The month is included as a factor to capture possible seasonal influences on prices

In [ ]:
# Get Consolidated Data
# https://etfdb.com/
# stock:  BLX.TO =  jan 2005, VMW = aug 2007
# S&P500: SPY = jan 1993
# gold:   GLD = nov 2004
# oil:    USO =apr 2006, OIL = apr 2011
# energy: PBD =  jun 2007 (clean), XLE= dec 1998 (oil & gas)
# gas:    BOIL=oct 2011
# vol:    VXX = jan 2009
# TSX:    EWC = mar 1996
# tech:   VGT (jan 2004-tech), QQQ (1999-nasdaq), SKYY (jul 2011 - cloud tech)

# VMW
config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},
          },
         'Technical':{
           'BBANDS':{'interval':'daily','time_period':20},
           'RSI':{'interval':'daily','time_period':14},
           'MACD':{'interval':'daily','time_period':None}
           },
         'Commodities':['GLD','OIL','SPY','VXX','QQQ','SKYY','VGT']
         }


# compact, full
stock = 'VMW'
outputsize = 'full'
df_vmw = get_consolidated_stock_data(stock,key,config,throttle=30,outputsize=outputsize,dropna=True)


# save to google drive
df_vmw.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
df_vmw.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.csv')


Complete:===>Ticker:VMW
Complete:===>Commodity:GLD
Complete:===>Commodity:OIL
Complete:===>Commodity:SPY
Complete:===>Commodity:VXX
Complete:===>Commodity:QQQ
Complete:===>Commodity:SKYY
Complete:===>Commodity:VGT
Complete:===>TREASURY_YIELD:5year
Complete:===>TREASURY_YIELD:10year
Complete:===>TREASURY_YIELD:30year
Complete:===>TREASURY_YIELD:3month
Complete:===>FEDERAL_FUNDS_RATE
Complete:===>NONFARM_PAYROLL
Complete:===>UNEMPLOYMENT
Complete:===>CONSUMER_SENTIMENT
Complete:===>INFLATION_EXPECTATION
Complete:===>BBANDS
Complete:===>RSI
Complete:===>MACD



Type                 Data                     Total    Missing   %      Action
-------------------  ---------------------  -------  ---------  ------  -----------
Ticker               VMW                       3550          0  0.0%    none
Ticker               GLD                       4238          0  0.0%    none
Ticker               OIL                       2621          0  0.0%    none
Ticker               SPY                 

In [ ]:
# VM Ware
plot_candlestick(df_vmw,2000)
plot_distribution(df_vmw,'close')
plot_technicals(df_vmw,2018)

In [ ]:
# Get Consolidated Data
# https://etfdb.com/
# stock:  BLX.TO =  jan 2005, VMW = aug 2007
# S&P500: SPY = jan 1993
# gold:   GLD = nov 2004
# oil:    USO =apr 2006, OIL = apr 2011
# energy: PBD =  jun 2007 (clean), XLE= dec 1998 (oil & gas)
# gas:    BOIL=oct 2011
# vol:    VXX = jan 2009
# TSX:    EWC = mar 1996
# tech:   VGT (jan 2004-tech), QQQ (1999-nasdaq), SKYY (jul 2011 - cloud tech)



# BLX.TO
config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},
          },
         'Technical':{
           'BBANDS':{'interval':'daily','time_period':20},
           'RSI':{'interval':'daily','time_period':14},
           'MACD':{'interval':'daily','time_period':None}
           },
         'Commodities':['GLD','OIL','SPY','PBD','XLE','EWC','VXX']
         }


# compact, full
stock = 'BLX.TO'
outputsize = 'full'
df_blx = get_consolidated_stock_data(stock,key,config,throttle=30,outputsize=outputsize,dropna=True)


# save to google drive
df_blx.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
df_blx.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.csv')

In [ ]:
# Boralex
plot_candlestick(df_blx,2000)
plot_distribution(df_blx,'close')
plot_technicals(df_blx,2018)

# Cypto Currencies

### Get Crypto Data
Pull crypto prices from Alpha Advantage

In [ ]:
# Not currently used
def get_crypto_intraday(symbol,key,interval='60min',outputsize='compact',market='USD'):
  """
  CRYPTO_INTRADAY
    symbol: DOGE, BTC, ETH
    key: apikey
    interval: 1min, 5min, 15min, 30min, 60min
    outputsize: compact(last 100), full (all)
    market: USD
  """

  # build query string, get data as json and convert to a dict
  url = f"https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol={symbol}&market={market}&interval={interval}&outputsize={outputsize}&apikey={key}"
  r = requests.get(url)
  d = r.json()

  # extract data to df
  df=pd.DataFrame(d[f'Time Series Crypto ({interval})']).T
  df.columns = ['open','high','low','close','volume']
  df['symbol'] = d['Meta Data']['2. Digital Currency Code']
  df['name'] = d['Meta Data']['3. Digital Currency Name']

  return df


In [ ]:
def get_crypto_data(symbol,key):
  """
  Pulls daily crypto prices from alpha advantage.
  Inputs:
    symbol: ETH, BTC, DOGE
    key:    The alpha advantage API key
  Output:
    a dataframe of crypto prices: open,high,low, close, volume
  """
  # build query string, get data as json and convert to a dict
  url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market=CAD&apikey={key}'
  r = requests.get(url)
  d = r.json()

  # extract data to df
  df=pd.DataFrame(d['Time Series (Digital Currency Daily)']).T

  # remove columns not required
  # returns the price in two currencies, just keep USD
  cols = [c for c in df.columns if '(CAD)' not in c]
  df=df.loc[:, cols]
  df.columns = ['open','high','low','close','volume','marketcap']
  df.drop(['marketcap'],axis=1,inplace=True)

  # change data types
  df.index = pd.to_datetime(df.index)

  # convert datatype to float
  for col in ['open','high','low','close','volume']:
    df[col] = df[col].astype('float')

  # add the cyrpto name
  df['symbol'] = d['Meta Data']['3. Digital Currency Name']

  return df
   

### Calculate Technical Indicators
+ Alpha Advantge does not calculate technical indicators for cyrpto
+ These functions replicate the calculations used for stocks


**References**
+ [Bolling Bands and RSI](https://www.analyticsvidhya.com/blog/2021/01/algotrading-using-technical-indicator-and-ml-models/)
+ [Moving Average Concergence Divergence (MACD)](https://towardsdatascience.com/implementing-macd-in-python-cc9b2280126a)



#### Bollinger Bands

In [ ]:
def calc_bollinger(df,feature,window=20,st=2):
  """
  Calculates bollinger bands for a price time-series.  Used for crypto currencies
  Input: 
    df     : A dataframe of time-series prices
    feature: The name of the feature in the df to calculate the bands for
    window : The size of the rolling window.  Defaults to 20 days with is standard
    st     : The number of standard deviations to use in the calculation. 2 is standard 
  Output: 
    Returns the df with the bollinger band columns added
  """

  # rolling mean and stdev
  rolling_m  = df[feature].rolling(window).mean()
  rolling_st = df[feature].rolling(window).std()

  # add the upper/lower and middle bollinger bands
  df['b-upper']  = rolling_m + (rolling_st * st)
  df['b-middle'] = rolling_m 
  df['b-lower']  = rolling_m - (rolling_st * st)


#### Relative Strength Index (RSI)

In [ ]:
def calc_rsi(df,feature='close',window=14):
  """
  Calculates the RSI for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
    window  : The size of the rolling window.  Defaults to 14 days which is standard
  Output: 
    Returns the df with the rsi band column added
  """
  # RSI
  # calc the diff in daily prices, exclude nan
  diff =df[feature].diff()
  diff.dropna(how='any',inplace=True)

  # separate positive and negitive changes
  pos_m, neg_m = diff.copy(),diff.copy()
  pos_m[pos_m<0]=0
  neg_m[neg_m>0]=0

  # positive/negative rolling means
  prm = pos_m.rolling(window).mean()
  nrm = neg_m.abs().rolling(window).mean()

  # calc the rsi and add to the df
  ratio = prm /nrm
  rsi = 100.0 - (100.0 / (1.0 + ratio))
  df['rsi']=rsi

#### Moving Average Convergence/Divergence (MACD)


In [ ]:
def calc_macd(df,feature='close'):
  """
  Calculates the MACD and signial for the input feature
  Input:
    df      : A dataframe with a time-series of prices
    feature : The name of the feature in the df to calculate the bands for
  Output: 
    Returns the df with the macd columns added
  """
  ema12 = df[feature].ewm(span=12,adjust=False).mean()
  ema26 = df[feature].ewm(span=26,adjust=False).mean()
  df['macd']=ema12-ema26
  df['macd_signal'] = df['macd'].ewm(span=9,adjust=False).mean()


### Combine Cyrpto Data
+ Daily Price/commodity data and monthly economic indicators from Alpha Advantage were consolidated and combined with calculated technical indicators
+ Some economic indicators were missing data.  Given that the data is time-series based, missing data points were linearly interpolated
+ The get_consolidated_crypto_data function performs these functions and returns a consolidated data frame free of missing data


In [ ]:
def get_consolidated_crypto_data(symbol,key,config,boll_window=20,boll_std=2,rsi_window=14,throttle=30,dropna=True):
  """
  Pulls data from alpha advantage and consolidates
  API Limitations: 5 API requests per minute and 500 requests per day
  Inputs:
    symbol: crypto ticker
    key   : api key
    config: dictionary which lists the economic indicators and commodities to pull
    throttle: number of seconds to wait between api requests
    dropna: True/False, drops any records with nan
  Output:
    A dataframe with consolidated price data for the symbol + economic/technical
    indicators and commodity prices
  """

  # Result header and accumulator
  header = ['Type','Data','Total','Missing',' % ','Action']
  summary =[]

  # Get crypto prices
  try:
    dff  = get_crypto_data(symbol,key)
    
    # add month feature
    dff['month'] = dff.index.month

    print(f'Complete:===>Crypto:{symbol}')

  except:
    print(f'Error:===>Crypto:{symbol}')


  # Get Commodity prices
  # ****************************************************************************
  for commodity in config['Commodities']:
    try:
      # get prices
      results = get_ticker_data(commodity,key,'full',throttle)
      df = results['data']
      summary.append(results['summary'])
      print(f'Complete:===>Commodity:{commodity}')


      # rename close to commodity name, remove unneeded columns and join with 
      # the stock prices by date
      df.rename(columns={'close':commodity},inplace=True)
      df.drop(['open','high','low','volume','symbol'],axis=1,inplace=True)
      dff = dff.join(df,how='left')
    except:
      print(f"Error===>Commodity:{commodity}")


  # Economic Indicators
  # ****************************************************************************
  # loop through the config to pull the requested data
  for indicator,values in config['Economic'].items():
    if indicator == 'TREASURY_YIELD':
      for tr in values:
        try:
          results = get_economic_indicators(indicator,key,interval=tr['interval'],maturity=tr['maturity'],throttle=throttle)
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}:{tr['maturity']}")

          df = results['data']
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": tr['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}:{tr['maturity']}")
   
    else: 
      # daily
      if values['interval']=='daily':
        try:
      
          results = get_economic_indicators(indicator,key,interval=values['interval'],throttle=throttle)
          df = results['data']
          summary.append(results['summary'])
          print(f"Complete:===>{indicator}")

          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")
  
      else: 
        try:
          # monthly or weekly
         
          results = get_economic_indicators(indicator,key,throttle=throttle)
          summary.append(results['summary'])
          df = results['data']
          print(f"Complete:===>{indicator}")

          # reindex to daily, fill missing values forward
          days = pd.date_range(start = min(df.index),end =max(df.index),freq='D')
          df =df.reindex(days,method = 'ffill')
      
          # join with the other data
          dff = dff.join(df,how='left')
          dff.rename(columns={"value": values['name']},inplace=True)
          dff.drop(['name', 'interval'], axis=1,inplace = True)
        except:
          print(f"Error===>{indicator}")

  # # Technical Indicators
  # ****************************************************************************
  calc_rsi(dff,'close',rsi_window)
  calc_bollinger(dff,'close',boll_window,boll_std)
  calc_macd(dff,'close')
      

  # Fill in any missing data after joining all datasets
  dff.fillna(method='bfill',inplace=True,axis = 0)

  # drop rows with missing commodity prices
  if dropna:
    dff.dropna(how='any',inplace=True)

  # print the results table
  print("\n\n")
  print(tabulate(summary,header))

  return dff

### Pull Crypto Data
A selection of economic and technical indicators were selected as possible inputs to the LSTM model. 
+ **Economic Indicators**
  + Interest Rates: The Fed interest rate, short, middle and long term bond yields
  + [Non-farm payrolls](https://www.investopedia.com/terms/n/nonfarmpayroll.asp) - the number of people currently employed in the US (monthly indicator)
  + [Unemployment rate](https://www.investopedia.com/terms/u/unemploymentrate.asp) - the percentage of people unemployed in the US
  + [Consumer sentiment](https://www.investopedia.com/terms/c/cci.asp) -measure of optimistic/pessimistic consumers based on survey data
  + [Inflation expection](https://fred.stlouisfed.org/series/MICH) -Median expected price change next 12 months, based on consumer surveys
+ **Commodities and Market Indexes**- ETFs are used as proxies
  + S&P500 Index (SPY ETF)
  + Gold Prices (GLD ETF)
  + Oil Prices (OIL ETF)
  + Energy Index (XLE ETF)
  + Gas Prices (BOIL ETF)
  + Market Volatility Index (VXX ETF)
+ **Technical Indicators**
  + Bollinger Bands
  + RSI (relative strenth indicator)
  + MACD (moving average convergence/divergence)
+ **Seasonal**
  + The month is included as a factor to capture possible seasonal influences on prices

In [ ]:
# Get Consolidated Data
# https://etfdb.com/
# cypto:  BTC, ETH,DOGE
# S&P500: SPY = jan 1993
# gold:   GLD = nov 2004
# oil:    OIL = apr 2011
# energy: XLE= dec 1998 (oil & gas)
# gas:    BOIL=oct 2011
# vol:    VXX = jan 2009

config ={'Economic':
         {'TREASURY_YIELD':[{'interval':'daily','maturity':'5year','name':'yield5y'},
                            {'interval':'daily','maturity':'10year','name':'yield10y'},
                            {'interval':'daily','maturity':'30year','name':'yield30y'},
                            {'interval':'daily','maturity':'3month','name':'yield3m'}
                            ],
          'FEDERAL_FUNDS_RATE':{'interval':'daily','name':'ir'},
          'NONFARM_PAYROLL':{'interval':'monthly','name':'nfp'},
          'UNEMPLOYMENT':{'interval':'monthly','name':'unemployment'},
          'CONSUMER_SENTIMENT':{'interval':'monthly','name':'cs'},
          'INFLATION_EXPECTATION':{'interval':'monthly','name':'infl'},
          },
         'Commodities':['GLD','OIL','SPY','XLE','VXX','BOIL']
         }

# compact, full
crypto = 'BTC'
df_btc = get_consolidated_crypto_data(crypto,key,config,throttle=30,dropna=True)



# scale the input and outputs
scaler_X = MinMaxScaler(feature_range=(0,1))
scaler_y = MinMaxScaler(feature_range=(0,1))
scaled_X = scaler_X.fit_transform(array_X)
scaled_y = scaler_y.fit_transform(array_y)


# save to google drive
df_btc.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data.pickle')
df_btc.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data.csv')

Complete:===>Crypto:BTC
Complete:===>Commodity:GLD
Complete:===>Commodity:OIL
Complete:===>Commodity:SPY
Complete:===>Commodity:XLE
Complete:===>Commodity:VXX
Complete:===>Commodity:BOIL
Complete:===>TREASURY_YIELD:5year
Complete:===>TREASURY_YIELD:10year
Complete:===>TREASURY_YIELD:30year
Complete:===>TREASURY_YIELD:3month
Complete:===>FEDERAL_FUNDS_RATE
Complete:===>NONFARM_PAYROLL
Complete:===>UNEMPLOYMENT
Complete:===>CONSUMER_SENTIMENT
Complete:===>INFLATION_EXPECTATION



Type                Data                     Total    Missing   %      Action
------------------  ---------------------  -------  ---------  ------  -----------
Ticker              GLD                       4238          0  0.0%    none
Ticker              OIL                       2621          0  0.0%    none
Ticker              SPY                       5507          0  0.0%    none
Ticker              XLE                       5507          0  0.0%    none
Ticker              VXX                       3182  

In [ ]:
# Bitcoin Prices & Economic Indicators
plot_candlestick(df_btc,2010)
plot_distribution(df_btc,'close')
plot_technicals(df_btc,2018)

# Feature Selection

## Feature Importance
A quantitative method was used to select key features to include in the LSTM model.  Recursive Feature Elimination (RFE) using a Random Forest regression model was used to calculate the importance and rank of each feature included in the model. 

The random forest estimator starts by using all available features, then eliminates the least influential features recursively until the desired number of features remains. The plot_feature_importance function calculates and plots the feature importance and ranks using RFE. 

In [ ]:
# load saved market data
stock = 'VMW'
df_vmw = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
print(f"\n{stock}")
print("="*60)
print(f"{stock}-all features: ",df_vmw.shape)

crypto='BTC'
df_btc = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{crypto}_market_data.pickle')
print(f"\n{crypto}")
print("="*60)
print(f"{crypto}-all features: ",df_btc.shape)

stock='BLX.TO'
df_blx = pd.read_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data.pickle')
print(f"\n{stock}")
print("="*60)
print(f"{stock}-all features: ",df_blx.shape)



VMW
VMW-all features:  (2570, 29)

BTC
BTC-all features:  (1000, 28)

BLX.TO
BLX.TO-all features:  (2550, 10)


In [ ]:
def plot_feature_importance(df,num_features,seed):
  """
  Calculate the feature importance and rank using RFE and random forest regression
  Input: 
    df: dataframe of features. Expects that the close price is the target variable
    num_features: the number of features to rank
    seed: set the seed for reproducability
  Output:
    A list of top features, and plos of feature importance and ranks
  """

  # get y target
  y = df.close

  # exclude price features and the symbol name from the feature list
  cols = [c for c in df.columns if c not in  ['symbol','close','open','high','low']]
  X = df[cols]


  # fit random forest model to get feature importance scores
  model = RandomForestRegressor(n_estimators=500, random_state=seed)
  model.fit(X, y)

  # use RFE to rank features
  rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=seed), n_features_to_select=num_features)
  fit = rfe.fit(X, y)

  # create plots
  fig = make_subplots(rows=1, cols=2,
                      subplot_titles = ('Feature Importance','Feature Ranks'),
                      shared_yaxes=True
  )

  # Print Top Features
  selected =[]
  for i in range(len(fit.support_)):
    if fit.support_[i]:
      selected.append(cols[i])
  
  print("Top Ranked Features")
  print("="*60)
  print(selected)
  
  # create the plot of feature importance
  fig.add_trace(go.Bar(
              name = 'Importance',
              x=model.feature_importances_,
              y=cols,
              orientation='h',
              marker=dict(color='skyblue'),
              opacity = 0.5
    ),row=1,col=1
  )

  # feature rank
  fig.add_trace(go.Bar(
      name = 'Rank',
      x = fit.ranking_,
      y= cols,
      orientation = 'h',
      marker=dict(color='crimson'),
      opacity = 0.5
    ),row=1,col=2
  )

  fig.update_layout(
      title = 'Feature Importance',
      template='plotly_white',
                    width=700,
                    height=800,
                    xaxis={'categoryorder':'total ascending'})

  fig.show()
  return selected


## Create Lag Features
The features may have more influence if they are lagged vs current.  For example, the unemployment rate from 40 days ago may influence the price to a greater extent than the current unemployment rate.  

To explore this possibility, the correlation between closing prices and current/lagged features are calculated and compared.  The number of days lagged was determined iteratively until the correlation with the largest magnitude was found.

The calc_optimal_feature_lags function calculates the correlation between each feature and the closing price. It loops over lags from zero days to 60 days and returns a list of features with higher higher correlations when lagged. 

The shift_features function adds a lagged version of the input features to the features data frame.

RFE is then used to select from all availble features (current and lagged) to find the most influential features to be included in the LSTM model.

In [ ]:
def calc_optimal_feature_lags(df,features_to_lag,lags):
  """
  Calculates the correlation between each feature and the closing price over a series of lags
  Inputs:
    df: A data frame of feartures.  Expects 'close' price to be included
    features_to_lag: A list of features to shift by each lag in lags
    lags: A list of lags (in days)
  Outputs:
    A dictionary of {feature name:lag (in days)}. Only returns features with 
    higher correlations with lags
  """
  # get a copy of the df
  dff = df.copy()

  # dict to store the optimal lag for each feature
  d = {}
  
  # loop each feature, each lag
  for feature in features_to_lag:
    # maximum abs correlation with close price, and the associated lag
    # reset after each feature
    max_corr = 0
    max_lag = 0
    for lag in lags:
      if feature != 'close':
        # name of shifted column: feature_shift_lag
        shift = f"_shift_{str(lag)}"

        # shift the feature by the lag,join with dff
        tshift=dff[feature].shift(lag)
        dff =dff.join(tshift,how='left',rsuffix=shift)

        # calc the corr with the close price
        corr = dff[['close',feature+shift]].corr()[feature+shift]['close']

        # update the max abs corr and associated lag
        if abs(corr)>max_corr:
          max_corr = corr
          max_lag = lag
    # save the max abs corr and associated lag
    if max_lag <0:
      d[feature] = max_lag

  return d

In [ ]:
def shift_features(df,features_shift):
  """
  Shifts features by time periods to convert them to lagged indicators
  Input:
    df: dataframe of features
    features_shift: dictionary  of {feature:period shift}
  Output:
    df: original dataframe + the shifted features
  """
  dff = df.copy()
  for feature,shift in features_shift.items():
    t_shift = pd.DataFrame(dff[feature].shift(periods=shift))
    dff =dff.join(t_shift,how='left',rsuffix='_shift')

  # remove nan introducted with lag features
  dff.dropna(how='any',inplace=True)

  return dff

#### Boralex (BLX.TO)
+ 10 features have higher correlations with the closing price of BLX when lagged by 60 days, and one has maximum correlation when lagged 40 days
+ The RFE process selected the 6 most important features to include in the model;
+ The current value of the clean energy index (PBD), the Bollinger bands(b-lower, b-middle,b-upper), the MACD indicator, and the 3-month bond yield 60 days before the current day
+ These 6 features are added to the close/open/high/low price features for inclusion in the LSTM model




In [ ]:
# Boralex
stock= df_blx['symbol'].unique().item()

# List of features to lag
# include economic and commodities (exclude technical indicators)
features_to_lag =[f for f in df_blx.columns if f not in ['open','high','low','symbol','b-lower','b-middle','b-upper','rsi','macd_hist','macd','macd_signal']]

# try shifting from zero to -60 days prior
lags = list(range(0,-70,-10))

# Find the lag with the largest absolute correlation with closing price
feature_shifts =calc_optimal_feature_lags(df_blx,features_to_lag,lags)
print(feature_shifts)

# shift the features according to 
df_blx_shifted = shift_features(df_blx,feature_shifts)

# plot the correlations
plot_corr(df_blx_shifted)

# plot feature importance and ranks
selected = plot_feature_importance(df_blx_shifted,6,1985)

# subset features, add back the price features
[selected.append(f) for f in ['open','close','high','low']] 
df_blx_features = df_blx_shifted[selected]

# # save to google drive
df_blx_features.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
df_blx_features.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.csv')



{'OIL': -60, 'XLE': -60, 'VXX': -60, 'yield5y': -60, 'yield10y': -60, 'yield30y': -60, 'yield3m': -60, 'ir': -60, 'unemployment': -60, 'cs': -40, 'infl': -60}


Top Ranked Features
['PBD', 'b-lower', 'b-middle', 'b-upper', 'macd', 'yield3m_shift']


#### VM Ware (VMW)
+ 10 features have higher correlations with VM Ware closing price when lagged   60 days 
+ The top-ranked features are a cloud-computing sector index('SKYY'), the 3-month bond yields 60 days before the current day, the Bollinger bands, and the RSI technical indicators
+ These 6 features are added to the close/open/high/low price features for inclusion in the LSTM model

In [ ]:
# VMW
stock= df_vmw['symbol'].unique().item()

# List of features to lag
# include economic and commodities (exclude technical indicators)
features_to_lag =[f for f in df_vmw.columns if f not in ['open','high','low','symbol','b-lower','b-middle','b-upper','rsi','macd_hist','macd','macd_signal']]

# try shifting from zero to -60 days prior
lags = list(range(0,-70,-10))

# Find the lag with the largest absolute correlation with closing price
feature_shifts =calc_optimal_feature_lags(df_vmw,features_to_lag,lags)
print(feature_shifts)

# shift the features according to 
df_vmw_shifted = shift_features(df_vmw,feature_shifts)

# plot the correlations
plot_corr(df_vmw_shifted)

# plot feature importance and ranks
selected = plot_feature_importance(df_vmw_shifted,6,1985)


# subset features, add back the price features
[selected.append(f) for f in ['open','close','high','low']] 
df_vmw_features = df_vmw_shifted[selected]

# save to google drive
df_vmw_features.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
df_vmw_features.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.csv')


{'volume': -60, 'OIL': -60, 'VXX': -60, 'yield5y': -60, 'yield10y': -60, 'yield30y': -60, 'yield3m': -60, 'ir': -60, 'unemployment': -60, 'cs': -60}


Top Ranked Features
['SKYY', 'b-lower', 'b-middle', 'b-upper', 'rsi', 'yield3m_shift']


#### Bitcoin (BTC)
+ 13 features have higher correlations with the current price of bitcoin when lagged
+ The RFE process selected the 6 most influential features to include in the model.  The S&P500 index level, the Bollinger band, and MSCD indicators, and the current 10-year bond yield.
+ These 6 features are added to the close/open/high/low prices features for inclusion in the LSTM model

In [ ]:
# Bitcoin
stock= 'BTC'

# List of features to lag
# include economic and commodities (exclude technical indicators)
features_to_lag =[f for f in df_btc.columns if f not in ['open','high','low','symbol','b-lower','b-middle','b-upper','rsi','macd_hist','macd','macd_signal']]

# try shifting from zero to -60 days prior
lags = list(range(0,-70,-10))

# Find the lag with the largest absolute correlation with closing price
feature_shifts =calc_optimal_feature_lags(df_btc,features_to_lag,lags)
print(feature_shifts)

# shift the features according to 
df_btc_shifted = shift_features(df_btc,feature_shifts)

# plot the correlations
plot_corr(df_btc_shifted)

# plot feature importance and ranks
selected = plot_feature_importance(df_btc_shifted,6,1985)


# subset features, add back the price features
[selected.append(f) for f in ['open','close','high','low']] 
df_btc_features = df_btc_shifted[selected]

# save to google drive
df_btc_features.to_pickle(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.pickle')
df_btc_features.to_csv(f'/content/drive/MyDrive/Colab Notebooks/data/{stock}_market_data_features.csv')


{'volume': -60, 'month': -60, 'GLD': -60, 'XLE': -60, 'VXX': -60, 'yield5y': -60, 'yield10y': -60, 'yield30y': -60, 'yield3m': -60, 'ir': -60, 'nfp': -60, 'unemployment': -60, 'cs': -60}


Top Ranked Features
['SPY', 'yield10y', 'b-upper', 'b-middle', 'b-lower', 'macd']


In [ ]:
#PCA
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler


# df_pca = df.copy()
# pca_cols =[c for c in df_pca.columns if df_pca[c].dtype != 'object']
# df_pca = df_pca[pca_cols]


# # scale
# scaler = StandardScaler()
# scaler.fit(df_pca)
# pca_X = scaler.transform(df_pca)
# # pca_y = scaler.transform(df_pca['close'])


# pca = PCA(n_components=0.90, svd_solver='full')
# pca.fit(pca_X)

# X = pca.transform(pca_X)

# print(pca.explained_variance_ratio_)
# X.shape